# <span style="color:#2834d4">loss function</span>

## 1. MSE(Mean Squared Error)

### $ MSE = \frac{1}{2}\sum_k (y_k - t_k)^2$

In [1]:
import numpy as np

In [2]:
def MSE_loss(y, t):
    return 0.5*np.sum((y-t)**2)

In [3]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

y1 = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
l1 = MSE_loss(np.array(y1), np.array(t))
print(l1)

y2 = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
l2 = MSE_loss(np.array(y2), np.array(t))
print(l2)

0.09750000000000003
0.5975


## 2. CEE(Cross Entropy Error)

### $ CEE = -\sum_k t_k \log{y_k} $

In [4]:
def CEE_loss(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y+delta))  # log 0 = -inf 

In [5]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

y1 = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
l1 = CEE_loss(np.array(y1), np.array(t))
print(l1)

y2 = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
l2 = CEE_loss(np.array(y2), np.array(t))
print(l2)

0.510825457099338
2.302584092994546


In [6]:
def CEE_loss(y, t):
    if y.ndim==1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_size = y.shape[0]
    
    return -np.sum(np.log(y[np.arrange(batch_size), t] + 1e-7)) / batch_size

```np.arrange(batch_size)``` : 0부터 batch_size-1 까지 넘파이 배열 생성 <br>
```y[np.arrange(batch_size), t]``` : ex) batch_size=3, t=[2, 7, 0] $\rightarrow$ [y[0,2], y[1,7], y[2,0]]

# <span style="color:#2834d4">수치 미분</span>

## 1. 미분

### 전방차분:  $\frac{df(x)}{dx} = \lim_{h \to 0}\frac{f(x+h) - f(x)}{h}$

- 수치 미분에는 오차가 포함됨.
- 오차를 줄이기 우해 중앙 차분을 쓰기도 함.
    - $(x+h)$와 $(x-h)$일때의 함수 $f$의 차분을 계산하는 방법
    
### 중앙차분: $\frac{df(x)}{dx} = \lim_{h \to 0}\frac{f(x+h) - f(x-h)}{2 \cdot h}$

In [7]:
def numerical_diff(f, x):
    h = 1e-4  # 0.0001
    return (f(x+h)-f(x-h)) / (2*h)

## 2. 편미분

### $f(x_0, x_1) = x_0^2 + x_1^2$

In [8]:
def function_2(x):
    return x[0]**2 + x[1]**2

#### $x_0=3, x_1=4$ 일 때, 편미분

In [9]:
def function_tmp1(x0):
    return x0*x0 + 4.0**2.0

print(numerical_diff(function_tmp1, 3.0))

def function_tmp2(x1):
    return 3.0**2.0 + x1*x1

print(numerical_diff(function_tmp2, 4.0))

6.00000000000378
7.999999999999119


# <span style="color:#2834d4">기울기</span>

In [10]:
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)  # x와 형상이 같은 값이 0으로 채워진 배열 생성
    
    for idx in range(x.size):
        tmp_val = x[idx]
        
        # f(x+h) 계산
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1-fxh2) / (2*h)
        x[idx] = tmp_val
    
    return grad

In [11]:
print("(3,4)에서의 기울기: ", numerical_gradient(function_2, np.array([3.0, 4.0])))
print("(0,2)에서의 기울기: ", numerical_gradient(function_2, np.array([0.0, 2.0])))
print("(3,0)에서의 기울기: ", numerical_gradient(function_2, np.array([3.0, 0.0])))

(3,4)에서의 기울기:  [6. 8.]
(0,2)에서의 기울기:  [0. 4.]
(3,0)에서의 기울기:  [6. 0.]


## 1. 경사하강법

## $x_i = x_i - \alpha \frac{\partial f}{\partial x_i}$

- $\alpha$ = learning rate : 매개변수 값을 얼마나 갱신하느냐를 정하는 것
    - 너무 크면 발산하고, 너무 작으면 학습이 느리게 진행

In [12]:
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f,x)
        x -= lr*grad 
    
    return x

#### $f(x_0, x_1) = x_0^2 + x_1^2$의 최솟값

In [13]:
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100)

array([-6.11110793e-10,  8.14814391e-10])

In [14]:
# lr이 너무 큰 경우
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=10.0, step_num=100)

array([-2.58983747e+13, -1.29524862e+12])

In [15]:
# lr이 너무 작은 경우
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=1e-10, step_num=100)

array([-2.99999994,  3.99999992])

## 2. 신경망에서의 기울기

In [16]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3)  # 정규분포로 초기화
        
    def predict(self, x):
        return np.dot(x, self.W)
    
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)
        
        return loss

In [17]:
net = simpleNet()
print(net.W)

[[ 0.92900301  1.5888279   0.12853715]
 [-0.5857252  -0.85550412 -0.44692251]]


In [18]:
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

[ 0.03024913  0.18334303 -0.32510797]


In [19]:
np.argmax(p)  # 최댓값의 인덱스

1

In [20]:
t = np.array([0, 0, 1])  # 정답 레이블
net.loss(x, t)

1.4083987510494458

In [21]:
f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)
print(dW)

[[ 0.20932481  0.24395434 -0.45327916]
 [ 0.31398722  0.36593151 -0.67991873]]


# <span style="color:#2834d4">학습 알고리즘 구현하기</span>

1. 미니배치
- 훈련 데이터 중 일부를 무작위로 가져오는데, 이렇게 선별한 데이터를 미니배치라 함.

2. 기울기 산출
- 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구함.
- 기울기는 손실 함수의 값을 가장 작게하는 방향을 제시함.

3. 매개변수 갱신
- 가중치 매개변수를 기울기 방향으로 아주 조금 갱신함.

4. 반복
- 1~3 단계 반복

## 1. 2층 신경망 클래스 구현하기

In [22]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient


class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
    
    # x: 입력 데이터, t: 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        acc = np.sum(y==t) / float(x.shape[0])
        
        return acc
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

In [23]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)

print(net.params['W1'].shape, net.params['b1'].shape) 
print(net.params['W2'].shape, net.params['b2'].shape)

(784, 100) (100,)
(100, 10) (10,)


In [24]:
x = np.random.rand(100, 784)
y = net.predict(x)

In [25]:
x = np.random.rand(100, 784)
t = np.random.rand(100, 10)

grads = net.numerical_gradient(x, t)
print(grads['W1'].shape, grads['b1'].shape)
print(grads['W2'].shape, grads['b2'].shape)

(784, 100) (100,)
(100, 10) (10,)


```np.random.rand(m,n)``` : 0~1 균일분포 표준정규분포 난수 mxn matrix 생성

```np.random.randn(m,n)``` : 평균이 0, 표준편차가 1인 가우시안 표준정규분포 난수 mxn matrix 생성

## 2. 미니배치 학습 구현하기

In [ ]:
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

train_loss_list = []
train_acc_list = []
test_acc_list = []

# hyperparameter
iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

iter_per_epoch = max(train_size / batch_size, 1)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

In [ ]:
# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

# <span style="color:#2834d4">정리</span>

- 기계학습에서 사용하는 데이터셋은 train과 test dataset으로 나눠 사용함.
    - train data로 학습한 모델의 범용 능력을 test data로 평가함.
- 신경망 학습은 손실 함수를 지표로, 손실 함수의 값이 작아지는 방향으로 가중치 매개변수를 갱신함.
    - 가중치 매개변수를 갱신할 때는 가중치 매개변수의 기울기를 이용하고, 기울어진 방향으로 가중치의 값을 갱신하는 작업을 반복함.
- 아주 작은 값을 주었을 때의 차분으로 미분하는 것을 수치 미분이라고 함.
    - 수치 미분을 이용해 가중치 매개변수의 기울기를 구할 수 있음.
    - 수치 미분을 이용한 계산은 시간이 걸리지만, 구현은 간단함.